In [1]:
!pip install wandb

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.8 MB 5.3 MB/s eta 0:00:01     |███████████████████████████▎    | 1.5 MB 5.3 MB/s eta 0:00:01
     |████████████████████████████████| 145 kB 8.8 MB/s eta 0:00:01
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21503 sha256=ffef2440240c6f85fe6c85171e4661d052c0dda5a9330a9f1791ba8ba929f6e0
  Stored in directory: /home/dhoof/.cache/pip/wheels/e1/e8/83/ddea66100678d139b14bc87692ece57c6a2a937956d2532608
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=674b6862a1d79984f5d3b58b3591e2f8bb1b83661f99766ecbdfbb9eb6fb177c
  Stored in directory: /home/dhoof/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built promise pathtools
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to

In [1]:
import itertools
import multiprocessing as mp
import os
import numpy as np
from datetime import datetime
import wandb
import glob
import json
import yaml

In [2]:
checkpoints=[
    {'dataset_trained_on':'cifar10','version': 0},
    {'dataset_trained_on':'cifar100','version': 0},
    {'dataset_trained_on':'grocerystore','version': 0},
    {'dataset_trained_on':'histaerial25x25','version': 0},
    {'dataset_trained_on':'histaerial50x50','version': 0},
    {'dataset_trained_on':'svhn','version': 0},
    {'dataset_trained_on':'tinyimagenet','version': 0},
            ]

# models_todo = ['lowres_resnet9','lowres_resnet50','lowres_densenet121','lowres_vgg16_bn']
models_todo = ['lowres_resnet9','lowres_resnet50','lowres_densenet121','lowres_vgg16_bn']
# models_todo = ['lowres_resnet9','lowres_resnet50']
data_todo = ['cifar10']
# data_todo = ['histaerial25x25']
# data_todo = ['cifar10']
gpus = [0,1,2,3]
workers_per_gpu = 1
queue_models_per_gpu=True

#!python pytorch-pretrained-cnns/train.py --classifier $model --dataset $dataset --batch_size 512 --gpu_id $gpu --num_workers 4 --scheduler "Step" --max_epochs 150 --checkpoints "last_best" --wandb $wandbname 1> /dev/null

# hparams_path_json = create_hparams(glob.glob(f"output/{checkpoint['dataset_trained_on']}/{model}/version_{checkpoint['version']}/hparams.yaml")[0])  
        # --params $hparams_path_json \

def gpu_worker(gpu, q):
    while not q.empty():
        (checkpoint, model, dataset) = q.get()    
        if queue_models_per_gpu:
            wandbname=("3005FineTuning"+str(model)).strip("_")
        else:
            wandbname=("2405CH"+str(checkpoint['dataset_trained_on'])+str(model)+str(dataset)).strip("_")
            
        checkpoint_name = glob.glob(f"output/{checkpoint['dataset_trained_on']}/{model}/version_{checkpoint['version']}/checkpoints/*.ckpt")[0]        
        !python pytorch-pretrained-cnns/train.py \
        --classifier $model \
        --dataset $dataset \
        --optimizer "adam" \
        --load_checkpoint $checkpoint_name \
        --replace_fc True \
        --learning_rate 1e-5 \
        --gpu_id $gpu \
        --num_workers 4 \
        --scheduler "None" \
        --max_epochs 20 \
        --batch_size 128 \
        --checkpoints "last_best" \
        --wandb $wandbname 1> /dev/null


if __name__ == '__main__':    
    if queue_models_per_gpu:
        if len(gpus) != len(models_todo):
            raise ValueError("Models have to be same amout of gpus")
        queues=dict()
        for gpu, model in zip(gpus,models_todo):
            queues[model]=(gpu,mp.Queue())        
        
        for checkpoint in checkpoints:
            for model_name in models_todo:
                for ds in data_todo:
                    queues[model_name][1].put((checkpoint, model_name, ds))        
        
        processes = []
        for model in models_todo:            
            p = mp.Process(target=gpu_worker, args=queues[model])
            p.start()
            processes.append(p)        
        
    else:
        que = mp.Queue()
        for checkpoint in checkpoints:
            for model_name in models_todo:
                for ds in data_todo:
                    que.put((checkpoint, model_name, ds))

        processes = []
        for gpu in gpus * workers_per_gpu:
            p = mp.Process(target=gpu_worker, args=(gpu, que))
            p.start()
            processes.append(p)

Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
wandb: Currently logged in as: dhoof. Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: dhoof. Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: dhoof. Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: dhoof. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.12.17 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.12.16
wandb: Run data is saved locally in ./output/cifar10/lowres_resnet9/version_1/wandb/run-20220601_124953-1
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run 1
wandb: ⭐️ View project at https://wandb.ai/dhoof/3005FineTuninglowres_resnet9
wandb: 🚀 View run at https://wandb.ai/dhoof/3005FineTuninglowres_resnet9/runs/1
wandb: wandb version 0.12.17 is available!  To upgrade, please run:
wandb:  $ pip install wandb --

In [5]:
for p in processes:
    p.kill()

In [2]:
!python pytorch-pretrained-cnns/train.py \
        --classifier "lowres_vgg16_bn" \
        --dataset "cifar10" \
        --load_checkpoint "output/tinyimagenet/lowres_vgg16_bn/version_0/checkpoints/epoch=133-step=52259.ckpt" \
        --replace_fc True \
        --learning_rate 1e-2 \
        --gpu_id 0 \
        --num_workers 8 \
        --scheduler "None" \
        --max_epochs 15 \
        --checkpoints "last_best"

Global seed set to 0
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Files already downloaded and verified
Files already downloaded and verified
Epoch 0    ━━━━━━━━━━━━━━━━━ 1/118 0:00:03 • -:--:-- 0.00it/s loss: 2.41 v_num: 
Epoch 0    ━━━━━━━━━━━━━━━━━ 2/118 0:00:03 • 0:00:09 12.94it/s loss: 2.41 v_num:
                                                               4 acc/train:     
Epoch 0    ━━━━━━━━━━━━━━━━━ 2/118 0:00:03 • 0:00:09 12.94it/s loss: 2.39 v_num:
                                                               4 acc/train:     
Epoch 0    ━━━━━━━━━━━━━━━━━ 3/118 0:00:03 • 0:00:09 13.09it/s loss: 2.39 v_num:
                                                               4 acc/train:     
Epoch 0    ━━━━━━━━━━━━━━━━━ 3/118 0:00:03 • 0:00:09 13.09it/s loss: 2.39 v_num:
Epoch 0    ╸━━━━━━━━━━━━━━━━ 4/118 0:00:03 • 0:00:10 12.60it/s loss: